In [1]:
from collections import Counter, defaultdict, namedtuple
from copy import deepcopy
import functools
import inspect
import json
import os
from pathlib import Path
import pickle
from pprint import pp, pprint, pformat
import re
import sys
import time
from typing import Dict, List

import numpy as np
import pandas as pd
import plotly.express as px

from colorutils import Color

from dotenv import load_dotenv
from jinja2 import Environment, FileSystemLoader, Template
import textwrap
from tqdm.auto import tqdm
# from tqdm import tqdm

import openai

from aic_nlp_utils.json import read_jsonl, read_json, write_json, write_jsonl, process_to_jsonl
from aic_nlp_utils.pycfg import parse_pycfg_args, read_pycfg
%load_ext autoreload
%autoreload 2

from prompt_opt.optimizers.predict_evaluate import get_candidate_score, rank_candidates
from prompt_opt.utils import *

sys.path.append("/home/drchajan/devel/python/FC/automated-fact-checking")

os.environ['VLLM_WORKER_MULTIPROC_METHOD']='spawn'
load_dotenv()

True

In [2]:
archive_jsonl = "/home/drchajan/devel/python/FC/prompt_opt/EXP/events-V3-mes3/seed_7223750/archive.jsonl"
archive = read_jsonl(archive_jsonl)

In [11]:
archive[0]["trn"]

[{'think': '**Step 1: Identify the Key Elements of the Query**\n\nFrom the given query, we can extract the following key elements:\n\n* Date of the event: Út, 22 úno 2022 16:21:00\n* Text describing the event: The text describes the European Union imposing sanctions on Russian representatives and banks, with Hungary being cautious about supporting the sanctions.\n* People involved in the event:\n\t+ Ursula von der Leyenová (ID: 2374)\n\t+ Charles Michel (ID: 1784)\n\t+ Viktor Orbán (ID: 6301)\n\t+ Vladimir Putin (ID: 1600)\n\n**Step 2: Analyze the Text to Extract Key Events and People**\n\nUsing NLP techniques such as NER and POS tagging, we can identify the key events and people mentioned in the text:\n\n* Key events:\n\t+ European Union imposing sanctions on Russian representatives and banks\n\t+ Hungary being cautious about supporting the sanctions\n* People involved in the events:\n\t+ Ursula von der Leyenová (ID: 2374)\n\t+ Charles Michel (ID: 1784)\n\t+ Viktor Orbán (ID: 6301)\n\

In [ ]:
archive_jsonl = "/home/drchajan/devel/python/FC/prompt_opt/EXP/sir_phrase-V3/seed_5373459/archive.jsonl"
archive = read_jsonl(archive_jsonl)

In [3]:
for c in archive:
    trn_len = len(c.get('trn', []))
    dev_len = len(c.get('dev', []))
    tst_len = len(c.get('tst', []))
    print(f'{c["id"]} ({c.get("parent_id")}) trn={trn_len}, dev={dev_len}, tst={tst_len}')

1 (None) trn=4, dev=8, tst=8
2 (None) trn=4, dev=8, tst=8
3 (None) trn=4, dev=8, tst=8
4 (None) trn=4, dev=8, tst=8
5 (None) trn=4, dev=8, tst=8
6 (None) trn=4, dev=8, tst=8
7 (None) trn=4, dev=8, tst=8
8 (None) trn=4, dev=8, tst=8
9 (None) trn=4, dev=8, tst=8
10 (None) trn=4, dev=8, tst=8
11 (4) trn=4, dev=8, tst=8
12 (4) trn=4, dev=8, tst=8
13 (4) trn=4, dev=8, tst=8
14 (4) trn=4, dev=8, tst=8
15 (4) trn=4, dev=8, tst=8
16 (10) trn=4, dev=8, tst=8
17 (10) trn=4, dev=8, tst=8
18 (10) trn=4, dev=8, tst=8
19 (10) trn=4, dev=8, tst=8
20 (10) trn=4, dev=8, tst=8
21 (15) trn=4, dev=8, tst=8
22 (15) trn=4, dev=8, tst=8
23 (15) trn=4, dev=8, tst=8
24 (15) trn=4, dev=8, tst=8
25 (15) trn=4, dev=8, tst=8
26 (14) trn=4, dev=8, tst=8
27 (14) trn=4, dev=8, tst=8
28 (14) trn=4, dev=8, tst=8
29 (14) trn=4, dev=8, tst=8
30 (14) trn=4, dev=8, tst=8
31 (30) trn=4, dev=8, tst=8
32 (30) trn=4, dev=8, tst=8
33 (30) trn=4, dev=8, tst=8
34 (30) trn=4, dev=8, tst=8
35 (30) trn=4, dev=8, tst=8
36 (29) trn=4,

In [ ]:
def candidate_info(archive, select_split, split, score_key):
    rank_idxs = rank_candidates(archive, split=select_split, score_key=score_key)
    print("rank indices:", rank_idxs)
    for ri in rank_idxs:
        c = archive[ri]
        select_score = get_candidate_score(c, split=select_split, score_key=score_key)
        test_score = get_candidate_score(c, split=split, score_key=score_key)
        print(f"score: {select_score:.3f}/{test_score:.3f}")
        print()
        prompt = candidate2prompt_md(c)
        pf(prompt)
        print()
        for ex in c[split]:
            gold = ex["gold"]
            pred = ex["pred"]
            query = ex["query"]
            lquery = query.lower()
            print(f' score: {ex["eval"][score_key]["score"]:.3f}')
            if isinstance(gold, str):
                print(" GOLD:", gold)
                print(" PRED:", pred)
            else:
                print("GOLD:")
                pp(gold)
                print("PRED:")
                pp(pred)
            # not_in_query = [p for p in pred if p.lower() not in lquery]
            # print(f"  not in query: {not_in_query}")
            print()
            pf(query)
            print()
        print("="*120)
    
# candidate_info(archive[:11], select_split="tst", split="tst", score_key="oa-07")
candidate_info(archive[99:], select_split="tst", split="tst", score_key="oa")

2024-12-11 06:21:07.294 | DEBUG    | prompt_opt.optimizers.predict_evaluate:get_candidate_score:129 - canidate id=100(89), dict_keys(['messages', 'parent_id', 'id', 'trn', 'dev', 'tst'])
2024-12-11 06:21:07.294 | DEBUG    | prompt_opt.optimizers.predict_evaluate:get_candidate_score:129 - canidate id=100(89), dict_keys(['messages', 'parent_id', 'id', 'trn', 'dev', 'tst'])
2024-12-11 06:21:07.295 | DEBUG    | prompt_opt.optimizers.predict_evaluate:get_candidate_score:129 - canidate id=100(89), dict_keys(['messages', 'parent_id', 'id', 'trn', 'dev', 'tst'])


rank indices: [0]
score: 0.497/0.497

**Query-to-Answer Transformation Instructions**

**Step 1: Identify Main Events**

* Read the query text and identify the most important events or actions.
* Use natural language processing (NLP) techniques to extract relevant information from the text.
* Prioritize events based on their relevance to the main topic.
* Include specific details such as dates, percentages, and statistics where relevant.
* Focus on the main event or action, rather than including additional details that are not relevant to the event.

**Step 2: Extract Relevant Details**

* Use NLP techniques to extract relevant details from the text, such as names of people involved in the events.
* Identify key entities involved in the events and include them in the event descriptions.
* Prioritize extracted details based on their relevance to the main events or actions.

**Step 3: Construct Event Descriptions**

* Use the extracted information from Steps 1 and 2 to construct a brief 

In [5]:
idx = 1
candidate = archive[idx]

In [6]:
candidate

{'messages': [{'role': 'system',
   'content': 'You are an AI assistant that uses a Chain of Thought (CoT) approach with reflection to respond to user input. Follow these steps:\n\n1. Think through the problem step by step; use the "Thinking" section to mark this stage.\n2. Reflect on your thinking to check for any errors or improvements using the "Reflection" section.\n3. Make any necessary adjustments based on your reflection.\n4. Provide your final, concise response in the "Response" section.\n\nImportant: The "Thinking" and "Reflection" sections are only for your internal reasoning process. \nDo not include any part of the final response in these sections. \nThe actual response to the user input must be entirely contained within the "Response" section.\n\nIt is absolutely CRITICAL that all your (the assistant\'s) outputs use Markdown containing exactly three consequent sections "Thinking", "Reflection", and "Response" as follows:\n\n# Thinking\n<Your step-by-step reasoning goes her

In [21]:
def get_candidate_score(candidate, split, score_key):
    sample_scores = [sample["eval"][score_key]['score'] for sample in candidate[split]]
    return np.mean(sample_scores)

def rank_candidates(candidates, split, score_key):
    candidate_scores = [get_candidate_score(candidate, split, score_key) for candidate in candidates]
    return np.argsort(candidate_scores, kind="stable")[::-1]
    
rank_candidates(archive, "tst", 'oa-07')

array([1, 0])

In [19]:
?np.argsort

Signature:       np.argsort(a, axis=-1, kind=None, order=None)
Call signature:  np.argsort(*args, **kwargs)
Type:            _ArrayFunctionDispatcher
String form:     <function argsort at 0x7fb6640d3920>
File:            ~/devel/python/FC/VENV/vllm/lib/python3.11/site-packages/numpy/core/fromnumeric.py
Docstring:      
Returns the indices that would sort an array.

Perform an indirect sort along the given axis using the algorithm specified
by the `kind` keyword. It returns an array of indices of the same shape as
`a` that index data along the given axis in sorted order.

Parameters
----------
a : array_like
    Array to sort.
axis : int or None, optional
    Axis along which to sort.  The default is -1 (the last axis). If None,
    the flattened array is used.
kind : {'quicksort', 'mergesort', 'heapsort', 'stable'}, optional
    Sorting algorithm. The default is 'quicksort'. Note that both 'stable'
    and 'mergesort' use timsort under the covers and, in general, the
    actual impleme